In [1]:
import pandas as pd
import pandas_datareader as pdr

uni = pd.read_clipboard()

prices = list()

for ticker in uni.squeeze():
    try:
        price = pdr.DataReader(ticker, "naver", start="1990-1-1")["Close"].astype(float)
        price.name = ticker
        prices.append(price)
    except Exception as exc:
        print(ticker, " does not exist.", exc)

prices = pd.concat(prices, axis=1)
prices.index = pd.to_datetime(prices.index)
prices = prices.sort_index()
prices.to_clipboard()

455340  does not exist. XML or text declaration not at start of entity: line 3, column 0
A426020  does not exist. XML or text declaration not at start of entity: line 3, column 0
A426030  does not exist. XML or text declaration not at start of entity: line 3, column 0


In [3]:
import os
import sys
from datetime import datetime, date, timedelta
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))

TODAY = date.today()
YESTERDAY = TODAY - timedelta(days=1)

def run_mlp_prediction() -> None:

    from reposit.vine.prediction import PredSettings
    from reposit.vine.prediction import BaggedMlpClassifier

    price_df = pd.read_excel("vine_universe.xlsx", sheet_name="Prices", index_col="Date", parse_dates=True).astype(float)

    pred_settings = PredSettings(
        train=True, save=False
    ).dict()
    prediction_model = BaggedMlpClassifier(**pred_settings)
    data = prediction_model.predict(price_df=price_df)

    data.to_clipboard()

run_mlp_prediction()

102110: 0.5462
310960: 0.5351
277630: 0.5468
232080: 0.6317
252000: 0.6119
228820: 0.5429
292160: 0.5555
310970: 0.5520
139260: 0.5018
315270: 0.5327
139290: 0.5648
139250: 0.5943
227540: 0.5105
227560: 0.4622
227550: 0.5866
139270: 0.5155
139220: 0.6021
139240: 0.6738
139230: 0.6411
139280: 0.5139
91220: 0.5081
157500: 0.5347
143860: 0.5083
91230: 0.5235
98560: 0.5208
261060: 0.5581
261070: 0.5258
138530: 0.5058
138520: 0.5769
138540: 0.5220
228800: 0.6001
228790: 0.4586
300610: 0.4571
305540: 0.7514
307510: 0.6988
307520: 0.5034
228810: 0.5791
157490: 0.6443
150460: 0.5506
289250: 0.5484
289260: 0.5657
289480: 0.5473
166400: 0.5871
211560: 0.5746
261140: 0.4858
210780: 0.5387
227570: 0.5415
217790: 0.5569
174350: 0.5331
147970: 0.5453
277640: 0.5562
277650: 0.5553
292150: 0.5144
157450: 0.5356
114820: 0.5442
302190: 0.5170
237440: 0.4945
272580: 0.5278
182490: 0.5314
245340: 0.5336
133690: 0.5466
192090: 0.5002
245360: 0.5495
117690: 0.5742
195920: 0.5463
241180: 0.5361
195930: 0.547

In [11]:
# PGA ETF allocation
import pandas_datareader as pdr
import pandas as pd

#allocations = pd.read_clipboard(index_col="ticker")
allocations = pd.read_excel("Factsheets Vertical.xlsm", sheet_name="2023-10", index_col='ticker').drop(columns='name')

navs = []
for allocation in allocations:

    weights = allocations[allocation]
    
    prices = []

    for asset, weight in weights.items():
        p = pdr.DataReader(asset.replace(".KS", ""), data_source="naver", start='1990-1-1')['Close'].astype(float)
        p.name = asset
        prices.append(p)
    prices = pd.concat(prices, axis=1)

    w = prices.loc["2023-9-27":] * weights.divide(prices.loc["2023-9-27"])
    n = w.sum(axis=1)
    n.name = allocation
    navs.append(n)

pd.concat(navs, axis=1).to_clipboard()

In [6]:
import pandas as pd
import pandas_datareader as pdr


def calculate_sample_benchmarks():
    RiskAllocation = {
        "5": {"069500": 0.70, "114260": 0.25, "153130": 0.05},
        "4": {"069500": 0.60, "114260": 0.35, "153130": 0.05},
        "3": {"069500": 0.50, "114260": 0.45, "153130": 0.05},
        "2": {"069500": 0.40, "114260": 0.50, "153130": 0.10},
        "1": {"069500": 0.30, "114260": 0.60, "153130": 0.10},
    }

    result = []

    for key, value in RiskAllocation.items():

        ds = []
        for k, v in value.items():
            d = pdr.DataReader(k, data_source="naver", start='1990-1-1')['Close'].astype(float)
            d.name = k
            ds.append(d)
    
        pri_return = pd.concat(ds, axis=1).dropna().pct_change().fillna(0)

        bm = pri_return.multiply(pd.Series(value)).sum(axis=1).add(1).cumprod()

        bm.name = f"PGA-BM-{key}"

        result.append(bm)

    result = pd.concat(result, axis=1)

    result = result.resample('D').last().ffill().stack().reset_index()

    result.columns = ['Date', 'Benchmark', 'Value']
    result["Market"] = "KR"
    return result


bm = calculate_sample_benchmarks()
bm.to_clipboard()

In [22]:
from xbbg import blp
import pandas as pd

uni = pd.read_clipboard().apply(lambda row: ' '.join(row), axis=1)

prices = list()

for ticker in uni.squeeze():
    print(ticker)
    try:
        price = blp.bdh(ticker, "px_last", start_date="1990-1-1").astype(float)
        price.name = ticker
        prices.append(price)
    except Exception as exc:
        print(ticker, " does not exist.", exc)

prices = pd.concat(prices, axis=1)
prices.index = pd.to_datetime(prices.index)
prices = prices.sort_index()
prices.to_clipboard()

5AH6145 KS Equity
5BS8137 KS Equity
5BG6960 KS Equity
3AL9835 KS Equity
1B38292 KS Equity
1B87901 KS Equity
1BD5217 KS Equity
1BD5373 KS Equity
1BF4184 KS Equity
1BT4394 KS Equity
1BT4410 KS Equity
1BT4436 KS Equity
1BT7306 KS Equity
1BT7348 KS Equity
1BT7389 KS Equity
1BT7421 KS Equity
1BV0968 KS Equity
1BX0164 KS Equity
1D20676 KS Equity
1D20874 KS Equity
1D58635 KS Equity
2BU0364 KS Equity
2BU6171 KS Equity
2BU6213 KS Equity
2C13374 KS Equity
2CD1743 KS Equity
2DK7856 KS Equity
4BU0297 KS Equity
4C12539 KS Equity
4CA6248 KS Equity
5B00871 KS Equity
5B30886 KS Equity
5B67003 KS Equity
5B67037 KS Equity
5B76772 KS Equity
5B78752 KS Equity
5B78794 KS Equity
5BA0563 KS Equity
5BC0579 KS Equity
5BG6820 KS Equity
5BG6945 KS Equity
5BH0939 KS Equity
5BH1010 KS Equity
5BS2064 KS Equity
5BS2072 KS Equity
5BS4466 KS Equity
5BS4490 KS Equity
5BS5059 KS Equity
5BS5067 KS Equity
5BS5075 KS Equity
5BS5406 KS Equity
5BS6008 KS Equity
5BS6255 KS Equity
5BS6263 KS Equity
5BS6271 KS Equity
5BS7188 KS

In [24]:
import os
import sys
from datetime import datetime, date, timedelta
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))

TODAY = date.today()
YESTERDAY = TODAY - timedelta(days=1)

def run_mlp_prediction() -> None:

    from core.strategy.mlpstrategy import PredSettings
    from core.analytics.prediction import BaggedMlpClassifier

    price_df = pd.read_excel("vine_universe.xlsx", sheet_name="Fund Prices", index_col="Date", parse_dates=True).astype(float)

    pred_settings = PredSettings(
        train=True, save=False
    ).dict()
    prediction_model = BaggedMlpClassifier(**pred_settings)
    data = prediction_model.predict(price_df=price_df)

    data.to_clipboard()

run_mlp_prediction()

5AH6145 KS Equity: 0.4884
5BS8137 KS Equity: 0.5238
5BG6960 KS Equity: 0.5205
3AL9835 KS Equity: 0.5405
1B38292 KS Equity: 0.5402
1B87901 KS Equity: 0.5798
1BD5217 KS Equity: 0.5332
1BD5373 KS Equity: 0.5384
1BF4184 KS Equity: 0.5370
1BT4394 KS Equity: 0.5249
1BT4410 KS Equity: 0.5256
1BT4436 KS Equity: 0.5271
1BT7306 KS Equity: 0.5249
1BT7348 KS Equity: 0.5243
1BT7389 KS Equity: 0.5257
1BT7421 KS Equity: 0.5212
1BV0968 KS Equity: 0.5091
1BX0164 KS Equity: 0.5428
1D20676 KS Equity: 0.5295
1D20874 KS Equity: 0.5228
1D58635 KS Equity: 0.4788
2BU0364 KS Equity: 0.5310
2BU6171 KS Equity: 0.5285
2BU6213 KS Equity: 0.5149
2C13374 KS Equity: 0.5250
2CD1743 KS Equity: 0.5244
2DK7856 KS Equity: 0.5169
4BU0297 KS Equity: 0.4947
4C12539 KS Equity: 0.5702
5B00871 KS Equity: 0.5277
5B30886 KS Equity: 0.5430
5B67003 KS Equity: 0.5392
5B67037 KS Equity: 0.5356
5B76772 KS Equity: 0.5388
5B78752 KS Equity: 0.5411
5B78794 KS Equity: 0.5444
5BA0563 KS Equity: 0.5059
5BC0579 KS Equity: 0.4921
5BG6820 KS E

In [25]:
# PGA ETF allocation
import pandas as pd

#allocations = pd.read_clipboard(index_col="ticker")
allocations = pd.read_excel("Factsheets Vertical.xlsm", sheet_name="2023-06_fund", index_col='ticker').drop(columns=['isin', 'name'])

navs = []
for allocation in allocations:

    weights = allocations[allocation]

    prices = []

    for asset, weight in weights.items():
        p = blp.bdh(asset, "px_last", start_date="1990-1-1").astype(float)
        p.name = asset
        prices.append(p)
    prices = pd.concat(prices, axis=1)
    prices.index = pd.to_datetime(prices.index)
    prices.columns = prices.columns.droplevel(1)
    
    w = prices.loc["2023-5-31":] * weights.divide(prices.loc["2023-5-31"])
    n = w.sum(axis=1)
    n.name = allocation
    navs.append(n)

pd.concat(navs, axis=1).to_clipboard()